In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import onnxmltools
import onnx
from onnxsim import simplify

In [ ]:
model = Sequential(
    [
        Dense(32, activation="relu", input_dim=100, name="main_input"),
        Dense(1, activation="sigmoid", name="dense_1"),
    ]
)


model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

# convert model to onnx
onnx_model = onnxmltools.convert_keras(model)

# convert model
model_simp, check = simplify(onnx_model)

assert check, "Simplified ONNX model could not be validated"

onnxmltools.utils.save_model(model_simp, "example.onnx")

In [ ]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from onnxsim import simplify

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

# Convert into ONNX format
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type, options={type(clr): {'zipmap': False}})
model_simp, check = simplify(onx)

with open("rf_iris.onnx", "wb") as f:
    f.write(model_simp.SerializeToString())

# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

pred_onx